<a href="https://colab.research.google.com/github/mroshan454/Replicating-ViT-Research-Paper/blob/main/Modular_Functions_For_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modular Code Structure for ViT Replication

This notebook explains the modular Python Scripts used in My Vision Transformer (ViT) Replication Project. Each Script is written to handle a specific part of the machine learning pipeline in a clear , reusable way - just like in the production codebases.


## 1. 📦`data_setup.py` - Dataset and Dataloader Builder

**Purpose** - This script helps in loading the image data into PyTorch `Dataloaders's`. It is written modularly so that it can be reused with any image classification dataset , simply by passing in the paths.



In [ ]:
%%writefile data_setup.py
"""
This file contains functionality for creating PyTorch DataLoader's for
image classification data
"""

import os
from torchvision import datasets,transforms
from torch.utils.data import dataloader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_data:str,
    test_data:str,
    transform:transforms.Compose,
    batch_size:int,
    num_workers:int=NUM_WORKERS
    ):
    """Creates training and testing DataLoaders.
    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and them into PyTorch Dataloader.

    Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the Dataloaders.
    num_workers: An integer for number of workers per DataLoader.

    Returns:
    A tuple of (train_dataloader,test_dataloader ,class_names).
    Where class_names is a list of the target classes.
    Example usage:
    train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=path/to/train_dir,
    test_dir = path/to/test_dir,
    transform=some_transform,
    batch_size=32,
    num_workers=4)
    """
    #Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir,transform=transform)
    test_data = datasets.ImageFolder(test_dir,transform=transform)
    #Get Class Names
    class_names = train_data.classes
    #Turn images into DataLoaders
    train_dataloader = DataLoader(train_data,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True)
    test_dataloader = Dataloader(test_data,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=num_workers,
                                 pin_memory=True)
    return train_dataloader , test_dataloader , class_names

## 2. ⚙️`engine.py` - Training and Evaluation Engine

This script contains all training and evaluation logic for the model. It include modular functions for:
* One-epoch training(`train_step`)
* One-epoch testing(`test_step`)
* Full training Pipeline over multiple epochs(`train`)

In [1]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch Model.
"""
from typing import Dict, List , Tuple

import torch

from tqdm.auto import tqdm

def train_step(model:torch.nn.Module,
               dataloader:torch.utils.Dataloader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device:torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    #Put the model in train mode
    model_train()

    #Setup train loss and training accuracy values
    train_loss , train_acc = 0, 0

    #Loop through the dataloader and batches
    for batch,(X,y) in enumerate(dataloader):
       #Send data to target device
       X, y = X.to(device) , y.to(device)

       #1. Forward Pass
       y_pred = model(X)

       #2. Calculate and accumulate the loss
       loss = loss_fn(y_pred,y)
       train_loss =+ loss.item()

       #3. Optimizer Zero Grad()
       optimizer.zero_grad()

       #4. Loss Backward
       loss.backward()

       #5. Optimizer Step
       optimizer.step()

       #Calculate and accumulate accuaracy metric across all batches
       y_pred_class = torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
       train_acc += (y_pred_class == y).sum().item()/len(pred)

    #Adjust the metrics to get average loss and accuarcy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc/ len(dataloader)

    return train_loss , train_acc

def test_step(model:torch.nn.Module,
              dataloader:torch.utils.Dataloader,
              loss_fn:torch.nn.Module,
              optimizer:torch.optim.Optimizer,
              device:torch.device) -> Tuple[float, float]:

    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    #Put the model in eval mode
    model.eval()

    #Setup test loss and test accuracy values
    test_loss , test_acc = 0,0

    # Turn on inference context manager
    with torch.inference_mode():
      #Loop through data batches
      for batch,(X,y) in enumerate(dataloader):
        #Send data to target device
        X , y = X.to(device), y.to(device)

        #1. Forward Pass
        test_pred_logits = model(X)

        #2. Calculate the loss
        loss = loss_fn(test_pred_logits,y)
        test_loss += loss.item()

        #Calculate and accumulate the accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    #Adjust the metrics to get average loss and accuracy per batch
    test_loss = test_loss/len(dataloader)
    test_acc = test_acc/len(dataloader)
    return test_loss , test_acc

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.Dataloader,
          test_dataloader:torch.utils.data.Dataloader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int,
          device:torch.device) -> Dict[str,List[float]]:

    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc=train_step()




Writing going_modular/engine.py


FileNotFoundError: [Errno 2] No such file or directory: 'going_modular/engine.py'